## Data preparation

In [1]:
# from google.colab import auth
# auth.authenticate_user()
import ee
ee.Authenticate()
ee.Initialize()
import tensorflow as tf
import folium
import numpy as np
from datetime import datetime, timedelta, date
import random

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xH1S53i_is0ZQCwvKzSdZl4wFdgpu3LqyLwh2p1RaTw&tc=TsUPlp5mNuwc21p_Gn2_ExijC-TfVbixb6qP2PR4vv4&cc=joxW6UkFxz26lX7am8kONJexkWieUywbr0HJItS7P4Q

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavddIrIdUw5ep2-RSuR1jVeOyUntTRxd_rQnOIV5dHUsPpyIeYERNypQ

Successfully saved authorization token.


In [2]:
# Cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(4).int()
    cloudsBitMask = ee.Number(2).pow(3).int()
    qa = image.select('QA_PIXEL')
    mask1 = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0))
    mask2 = image.mask().reduce('min')
    # mask3 = image.select(opticalBands).gt(0).And(
    #     image.select(opticalBands).lt(10000)).reduce('min')
    mask = mask1.And(mask2)  # .And(mask3)
    return image.updateMask(mask)
# Cloud masking function for s2


def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)


def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))


def merge_s2_l8(s2, l8):
    merged = ee.ImageCollection([s2, l8]).mean()
    return merged


def s2_vi(s2):
    s2 = s2.addBands(s2.normalizedDifference(['nir', 'red']).rename('ndvi'))
    s2 = s2.addBands(s2.select(['ndvi']).multiply(
        s2.select(['nir'])).rename('nirv'))
    return s2


In [3]:
# wheat is 24, sub 24 with 1 then set all other types as zerop(background)
def filter_type(img):  # TODO refactor for better generalization
    img = img.expression('cropland = cropland == 1? 255 : cropland',
                         {'cropland': img.select('cropland')})
    img = img.expression('cropland = cropland == 24? 1 : cropland',
                         {'cropland': img.select('cropland')})
    img = img.expression('cropland = cropland > 1? 0 : cropland',
                         {'cropland': img.select('cropland')})

    return img


In [4]:
def indexJoin(collectionA, collectionB, propertyName):
    joined = ee.ImageCollection(ee.Join.saveFirst(propertyName).apply(
        primary=collectionA,
        secondary=collectionB,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        )
    ))
    # Merge the bands of the joined image.
    return joined.map(lambda image: image.addBands(ee.Image(image.get(propertyName))))


In [7]:
us = ee.FeatureCollection(
    "FAO/GAUL_SIMPLIFIED_500m/2015/level1").filter('ADM0_NAME == "United States of America"')
states = ['Washington','Oregon','Idaho','Montana','Kansas','South Dakota','Nebraska','Colorado','Oklahoma','Texas']  # 'Kansas','South Dakota','Nebraska'
trainingPolys = us.filter(ee.Filter.inList("ADM1_NAME", states))


## Select bands and set the train image kernelsize (how big the train image is)

In [8]:
"""# Data Preparation"""
FOLDER = 'training_us_foraus_wheat'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Define year

In [9]:
bandNamesOut = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
bandNamesl8 = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
bandNamesS2 = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']


In [10]:
BANDS = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'ndvi', 'nirv']
RESPONSE = 'cropland'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]


In [11]:
trainingPolys = us.filter(ee.Filter.inList("ADM1_NAME", states))
trainingPolysList = trainingPolys.toList(trainingPolys.size())


In [12]:
MAX_CLOUD_PROBABILITY = 65
l8sr = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').map(maskL8sr)
l8sr = l8sr.select(bandNamesl8, bandNamesOut)

s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').map(maskEdges)


In [13]:
BANDS = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'ndvi', 'nirv']
RESPONSE = 'cropland'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]


In [ ]:
def histo(im, geometry):
    _ = im.reduceRegion(
        reducer=ee.Reducer.histogram(),
        geometry=geometry,
        scale=500,
        maxPixels=1e13)
    featurestats = ee.Feature(None, _)
    states = ee.FeatureCollection([featurestats])
    return states


In [18]:
for YEAR in range(2019, 2020):
    dte1 = date(YEAR, 5, 1)
    l8_dte1 = date(YEAR, 5, 31)
    s2_dte1 = date(YEAR, 5, 31)
    l8_criteria = ee.Filter.date(dte1.strftime(
        '%Y-%m-%d'), l8_dte1.strftime('%Y-%m-%d'))
    s2_criteria = ee.Filter.date(dte1.strftime(
        '%Y-%m-%d'), s2_dte1.strftime('%Y-%m-%d'))
    l8_reduced = l8sr.filter(l8_criteria).median().multiply(
        0.0000275).add(-0.2).float()

    s2SrWithCloudMask = indexJoin(s2Sr.filter(
        s2_criteria), s2Clouds.filter(s2_criteria), 'cloud_mask')
    s2_reduced = s2SrWithCloudMask.map(maskClouds)
    s2_reduced = s2_reduced.select(
        bandNamesS2, bandNamesOut).median().divide(10000).float()

    image = merge_s2_l8(s2_reduced, l8_reduced)
    image = s2_vi(image).float()

    dte2 = date(YEAR, 6, 1)
    l8_dte2 = date(YEAR, 6, 30)
    s2_dte2 = date(YEAR, 6, 30)
    l8_criteria = ee.Filter.date(dte1.strftime(
        '%Y-%m-%d'), l8_dte2.strftime('%Y-%m-%d'))
    s2_criteria = ee.Filter.date(dte1.strftime(
        '%Y-%m-%d'), s2_dte2.strftime('%Y-%m-%d'))
    l8_reduced = l8sr.filter(l8_criteria).median().multiply(
        0.0000275).add(-0.2).float()

    s2SrWithCloudMask = indexJoin(s2Sr.filter(
        s2_criteria), s2Clouds.filter(s2_criteria), 'cloud_mask')
    s2_reduced = s2SrWithCloudMask.map(maskClouds)
    s2_reduced = s2_reduced.select(
        bandNamesS2, bandNamesOut).median().divide(10000).float()

    image1 = merge_s2_l8(s2_reduced, l8_reduced)
    image1 = s2_vi(image1).float()
    image2 = image1.subtract(image)
    image_with_different = image.addBands(image1).addBands(image2)
    image = image.addBands(image1)

    YOI = date(YEAR, 1, 1).strftime('%Y-%m-%d')
    cdl = ee.ImageCollection('USDA/NASS/CDL').filter(
        ee.Filter.date(YOI, '2021-12-31')
    ).select('cropland').map(filter_type).first()
    featureStack = ee.Image.cat([
        image,
        cdl.select(RESPONSE)
    ]).float()

    list1 = ee.List.repeat(1, KERNEL_SIZE)
    lists = ee.List.repeat(list1, KERNEL_SIZE)
    kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

    arrays = featureStack.neighborhoodToArray(kernel)
    # These numbers determined experimentally.
    n = 100  # Number of shards in each polygon.
    N = 1000  # Total sample size in each polygon.

    i = random.randint(0, 100)

    for g in states:
        geomSample = ee.FeatureCollection([])
        histo_states = ee.FeatureCollection([])
        trainingPoly = trainingPolys.filter(ee.Filter.eq("ADM1_NAME", g))
        # histo_states = histo(image1,trainingPoly)
        for j in np.arange(100):
            sample = arrays.sample(
                region=trainingPoly,
                numPixels=3,  # Size of the shard.
                seed=i,
                scale=30,
                tileScale=8
            )

            i = i+1
            geomSample = geomSample.merge(sample)

        desc = TRAINING_BASE + '_' + \
            str(g) + '_' + str(YEAR) + '_' + str(KERNEL_SIZE)
        # task1 = ee.batch.Export.table.toDrive(
        #                   folder='RDA',
        #                   collection=histo_states,
        #                   description=str(YEAR)+'_histogram_'+g,
        #                   fileFormat='CSV'
            #  )    
        task = ee.batch.Export.table.toDrive(
            folder=FOLDER,
            collection=geomSample,
            description=desc,
            selectors=list(image.bandNames().getInfo()) + ['cropland'],
            fileFormat='TFRecord',
        )
        # task1.start()
        task.start()

    # i=random.randint(0,100)
    # geomSample = ee.FeatureCollection([])
    # g = 'Colorado'
    # trainingPoly = trainingPolys.filter(ee.Filter.eq("ADM1_NAME",g))
    # for j in np.arange(1000):
    #   trainingPoly = us.filter(ee.Filter.eq("ADM1_NAME",g))
    #   sample = arrays.sample(
    #                   region=trainingPoly,
    #                   numPixels=3,  # Size of the shard.
    #                   seed=i,
    #                   scale= 30,
    #                   tileScale=8
    #                   )
    #   i = i+1
    #   geomSample = geomSample.merge(sample)

    # desc = EVAL_BASE + '_' + str(g)+ '_' + str(YEAR)+ '_' + str(KERNEL_SIZE)
    # task = ee.batch.Export.table.toDrive(
    #     folder= FOLDER,
    #     collection=geomSample,
    #     description=desc,
    #     selectors =  list(image.bandNames().getInfo()) + ['cropland'],
    #     fileFormat='TFRecord',
    # )
    # task.start()


## Export task

In [ ]:
from pprint import pprint
pprint(ee.batch.Task.list())


## Export prediciton